In [ ]:
import pandas as pd
import glob
import os
import re
import matplotlib.pyplot as plt

# Only include result CSVs, not the water content file
csv_files = [
    f for f in glob.glob(r"C:\Users\Kilian\Documents\LeafH2O\measurements\250522\results\*.csv")
    if "leaf_water_content" not in os.path.basename(f).lower()
]
dfs = [pd.read_csv(f) for f in csv_files]
df = pd.concat(dfs, ignore_index=True)

def parse_filename(fname):
    base = os.path.basename(fname)
    match = re.match(r"(\d+)Meas_(\d+m).*_(\w+)\\.las", base)
    if match:
        meas, dist, leaf = match.groups()
        return meas, dist, leaf
    parts = base.split('_')
    if len(parts) >= 3:
        if parts[0].endswith('Meas'):
            meas_num = parts[0][:-4] if parts[0][:-4].isdigit() else parts[0]
        else:
            meas_num = parts[0]
        return meas_num, parts[1], parts[-1].replace('.las', '')
    return None, None, None

df[['measurement', 'distance', 'leaf_kind']] = df['filename'].apply(
    lambda x: pd.Series(parse_filename(x))
)

pivot = df.pivot_table(
    index=['leaf_kind', 'measurement'],
    columns='distance',
    values='mean_intensity_corr'
).reset_index()

pivot.to_csv('combined_by_leaf_and_distance.csv', index=False)


###### ADD leaf water content file ######


# Read the CSV, skipping the first 4 rows (adjust if needed)
lwc = pd.read_csv(
    r'C:\Users\Kilian\Documents\LeafH2O\measurements\250522\leaf_water_content.csv',
    sep=';',
    skiprows=4,
    header=None,
    encoding="latin1"
)

# The file has 11 columns, so set 11 column names
lwc.columns = [
    'leaf_kind', 'before1', 'after1', 'before2', 'before3', 'after_end1', 'after_end2', 'leaf_area', 'empty1', 'empty2', 'empty3'
]

# Keep only relevant columns
# lwc = lwc[['leaf_kind', 'before1', 'before2', 'before3']]
# Clean up leaf_kind for matching
lwc['leaf_kind'] = lwc['leaf_kind'].str.lower().str.replace(' ', '').str.replace('ß', 'ss').str.replace('', 'ss')

# Manual mapping from lwc leaf_kind to reflectance leaf_kind
leaf_kind_map = {
    'ahornxxl': 'ahornxxl',
    'Ahorn gross': 'ahorn-gross',
    'Ahorn klein': 'ahorn-klein',
    'linde I': 'linde1',
    'Linde II': 'linde2',
    'Linde III': 'linde3',
    'Linde links': 'linde-links',
    'Linde rechts': 'linde-r',
    # Add more mappings as needed
}
# Apply mapping (after lower/strip/replace)
lwc['leaf_kind'] = lwc['leaf_kind'].replace(leaf_kind_map)

# Melt the lwc DataFrame to long format for easier merging
lwc_long = lwc.melt(id_vars=['leaf_kind'], var_name='weight_measurement', value_name='weight')
# Map 'before1' -> '1', 'before2' -> '2', 'before3' -> '3'
lwc_long['measurement'] = lwc_long['weight_measurement'].str.extract(r'before(\d)').astype(str)
lwc_long = lwc_long.drop(columns=['weight_measurement'])

# Merge with pivot DataFrame on leaf_kind and measurement
combined = pd.merge(pivot, lwc_long, on=['leaf_kind', 'measurement'], how='left')


# Convert weight to float (replace comma with dot)
combined['weight'] = combined['weight'].str.replace(',', '.').astype(float)

# Save combined DataFrame
combined.to_csv('combined_reflectance_weight.csv', index=False)

# Plotting
fig, ax1 = plt.subplots(figsize=(10, 6))

# X positions for each row
x = range(len(combined))

print(combined)


: 

In [ ]:

# Plot reflectance (mean intensity) for 10m and 27m
ax1.plot(x, combined['10m'], marker='o', label='Mean Intensity 10m', color='blue')
ax1.plot(x, combined['27m'], marker='o', label='Mean Intensity 27m', color='green')
ax1.set_xlabel('Leaf Kind / Measurement')
ax1.set_ylabel('Mean Intensity')
ax1.set_xticks(x)
ax1.set_xticklabels([f"{row['leaf_kind']}-{row['measurement']}" for _, row in combined.iterrows()], rotation=45)
ax1.legend(loc='upper left')

# Second y-axis for weight
ax2 = ax1.twinx()
ax2.plot(x, combined['weight'], marker='s', color='red', label='Weight (before measurement)')
ax2.set_ylabel('Weight (g)')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()